In [5]:
import pandas as pd
import numpy as np
import json
import os

ANGLE_W = 1
SPEED_W = 15

In [6]:
# Return df with added movement, angle and time change data for each hand
# Each row is relative to the previous note on that hand
def hand_stats(notes: pd.DataFrame):
    # Calculate note centers
    notes['_yCenter'] = 1 + notes['_lineLayer'] * 0.55
    notes['_xCenter'] = -0.9 + notes['_lineIndex'] * 0.6

    # Split to left and right handed notes
    left = notes[notes['_type'] == 0].copy()
    right = notes[notes['_type'] == 1].copy()

    # Calculate position and angle changes between notes
    # Each row contains the change between the previous note and itself
    for df in [left, right]:
        df['_xMovement'] = df['_xCenter'].diff()
        df['_yMovement'] = df['_yCenter'].diff()
        df['_totMovement'] = np.hypot(df['_xMovement'], df['_yMovement'])
        df['_angleChange'] = np.arctan(df['_yMovement'] / df['_xMovement'])
        df['_timeChange'] = df['_time'].diff()
        df.iloc[0] = df.iloc[0].fillna(0) # Set movements and changes to 0 in first row

    # Recombine dfs now that we've calculated the hand specific stats
    notes = pd.concat([left, right]).sort_index()

    return notes

In [7]:
def calculate_complexity(notes: pd.DataFrame, mapset_info):
    # Convert note timings from beats to seconds
    notes['_time'] = notes['_time'] * (1 / mapset_info['_beatsPerMinute']) * 60

    # Add hand stats
    notes = hand_stats(notes)

    # Calculate overall angle and swing speed stats
    average_angle = notes['_angleChange'].abs().mean()
    sum_of_swing_times = notes[notes['_timeChange'] < 5]['_timeChange'].sum()
    total_distance = notes[notes['_totMovement'] > 0.54]['_totMovement'].sum()
    average_speed = total_distance / sum_of_swing_times

    # Complexity!
    complexity = ANGLE_W * average_angle + SPEED_W * average_speed

    return complexity

In [8]:
root = './accsaber-maps'

for entry in os.scandir(root):
    if not os.path.isdir(entry):
        continue
    
    files = [f for f in os.listdir(entry) if os.path.isfile(os.path.join(entry, f))]
    info_path = ''
    if 'Info.dat' in files:
        info_path = os.path.join(entry, 'Info.dat')
    elif 'info.dat' in files:
        info_path = os.path.join(entry, 'info.dat')
    else:
        continue
    
    with open(info_path, encoding='utf8') as json_data:
        mapset_info = json.load(json_data)
    
    standard_characterisitic = next((x for x in mapset_info['_difficultyBeatmapSets'] if x['_beatmapCharacteristicName'] == 'Standard'))
    for diff in standard_characterisitic['_difficultyBeatmaps']:
        diff_file_name = diff['_beatmapFilename']
        if diff_file_name in files: # Assumes only ranked diffs are in the directory
            diff_path = os.path.join(entry, diff_file_name)
            with open(diff_path) as json_data:
                diff_data = json.load(json_data)
            
            notes = pd.DataFrame(diff_data['_notes'])
            print(f'{mapset_info["_songName"]} - {diff["_difficulty"]} - {calculate_complexity(notes, mapset_info)}')

Hocus Pocus - Expert - 34.615435306122244
The Rest Of Life - Normal - 4.0095914192989826
All Eyes On Me - Hard - 18.252537783025527
Sleigh Ride - Easy - 8.30917052591243
Pastel Rain - Hard - 21.10957995076821
Impatient - Normal - 12.72862723013272
Ashes - Easy - 6.887826920191303
Just the Way You Are - Normal - 20.604070521902024
Climax - Easy - 22.34119308929765
Face Down - Hard - 30.841960787487107
Hellcat - Hard - 27.056257615081677
Pika Girl - Hard - 24.815457281410556
Step! - Easy - 10.036260446809948
Kakurenbocchi - Hard - 27.351885542589844
We are human beings - Normal - 23.92492908433902
Robber And Bouquet - Hard - 28.70329998586496
Swim - Expert - 27.127590819841316
Die For You - Expert - 21.819364741678594
MORE - Hard - 19.702520930070165
RGB - Expert - 27.504618195597185
2U - Easy - 4.486114489730842
24K Magic - Easy - 8.734619037625023
That's What I Like - Hard - 18.073206433753196
Finesse (Remix) - Easy - 7.656541198125073
Peanut Butter Jelly - Normal - 12.925040993807537
